In [1]:
import requests
import sys
from idxutils import gen_in_network_links
from mrfutils_jsonopen_async import InvalidMRF, JSONOpen
from tqdm import tqdm
import gzip
import aiocsv
import aiofiles
import asyncio
import aiohttp

In [2]:
filename = "F:\\_Bounty\\2022-12-01_anthem_index.json"

In [3]:
async def get_file(url, session):  
    async with JSONOpen(url) as f:
        print(type(f))
        print(dir(f))
        string = f.read()
    async with session.head(url) as r:
        size = r.headers.get("Content-Length")
    return url, string, size
    # if "Nevada" in f.read(100).decode("utf-8"):
    #     print("\nFound:", url)
    # return



In [4]:
async def main():
    async with aiofiles.open("delete.csv", "a", newline="") as output_csv:
        writer = aiocsv.AsyncDictWriter(output_csv, fieldnames=["url", "string", "size"])
        # writer.writeheader()
        connector = aiohttp.TCPConnector(limit_per_host=10)
        bar = tqdm(total=61996706)
        async with aiohttp.ClientSession(connector = connector, auto_decompress=False) as session:
            for in_network_file in gen_in_network_links(filename):
                
                # if i < 66300:
                #     i+=1
                #     continue
                url, string, size = await get_file(in_network_file, session)
                await writer.writerow({
                    "url": url,
                    "string": string.replace("\n", "").replace('{""reporting_entity_name"":', "").replace("\r", ""),
                    "size": size
                })

                bar.update(1) 

In [5]:
t = await main()

  0%|          | 0/61996706 [00:00<?, ?it/s]

AttributeError: __enter__